In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
ds_api_key = os.getenv("DEEP_SEEK_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_openai import ChatOpenAI
deepseek = ChatDeepSeek(
    api_key=ds_api_key,
    model="deepseek-chat"
)

openai = ChatOpenAI(
    api_key=openai_api_key,
    base_url="https://yunwu.ai/v1",
    model_name="gpt-5-mini",
)

In [ ]:
from langchain_core.tools import tool
import requests

import requests
from typing import List, Dict

def web_search_tool_kit():
    # 建议将 URL 外部化或作为配置
    searx_url = "https://sousuo.emoe.top/search"
    crawl4ai_url = "http://localhost:11235/md"

    @tool(parse_docstring=True)
    def web_search(query: str, max_results: int = 5, categories: str = "general", language: str ="zh-CN", engine: str | None = None) -> str:
        """
        利用 SearXNG 引擎进行互联网搜索。适用于获取实时新闻、技术文档或百科知识。
        
        Args:
            query (str): 具体的搜索关键词。
            max_results (int): 期望返回的结果条数，默认为 5。
            categories (str): 搜索类别。可选值: 'general', 'it', 'science', 'news', 'images', 'videos'。
            language (str): 搜索语言。默认为 "zh-CN"。
            engine (str | None): 指定使用的搜索引擎名称，如 "google", "bing" 等。默认为 None，表示使用默认引擎。
        
        Returns:
            str: 格式化的搜索结果列表，每条包含标题、来源链接和内容摘要。
        """
        params = {
            "q": query,
            "format": "json",
            "engine": engine,
            "categories": categories,
            "language": language,
        } if engine else {
            "q": query,
            "format": "json",
            "categories": categories,
            "language": language,
        }
        
        try:
            response = requests.get(searx_url, params=params, timeout=15)
            response.raise_for_status()
            raw_results = response.json().get("results", [])
        except Exception as e:
            return f"搜索失败: {str(e)}"

        # --- 核心优化：数据清洗 ---
        processed_results = []
        # 只取前 max_results 条，避免 Token 溢出
        for res in raw_results[:max_results]:
            # 提取 AI 需要的关键信息
            title = res.get("title", "无标题")
            link = res.get("url", "无链接")
            snippet = res.get("content", "无描述")
            
            # 格式化为易于 AI 阅读的字符串
            processed_results.append(f"标题: {title}\n链接: {link}\n摘要: {snippet}\n---")
            
        if not processed_results:
            return "未找到相关结果。"
            
        return "\n".join(processed_results)
    
    
    @tool(parse_docstring=True)
    def web_reader(url: str, priority: int = 1) -> str:
        """
        当你需要阅读特定网页的详细内容时使用此工具。
        支持动态加载的网页（如单页应用）。

        Args:
            url (str): 要读取的完整网页 URL。
            priority (int): 爬取优先级，1-10 之间。
        
        Returns:
            str: 网页的正文内容（Markdown 格式）。
        """
        payload = {
            "url": url,
            "f": "fit"
        }
        
        try:
            response = requests.post(crawl4ai_url, json=payload, timeout=30)
            response.raise_for_status()
            
            data = response.json()

            if data.get("success") and data.get("markdown"):
                content = data.get("markdown", "")
                if len(content) > 5000:
                    return content[:5000] + "\n\n(内容过长，已自动截断...)"
                return content
            else:
                return f"未能提取内容: {data.get('error', '未知错误')}"
                
        except Exception as e:
            return f"读取网页失败: {str(e)}"
        
    return [web_search, web_reader]

def commomon_tool_kit():

    @tool
    def now():
        """获取当前的日期和时间。"""
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    return [now]

def langfuse_tool_kit():

    @tool
    def get_prompt(name : str) -> str:
        pass

    @tool
    def list_prompts() -> List[str]:
        pass

    @tool
    def delete_prompt(name: str) -> str:
        pass
    return [get_prompt, list_prompts, delete_prompt]

In [ ]:
from deepagents import create_deep_agent
from langfuse.langchain import CallbackHandler
ds_agent = create_deep_agent(
    name="OpenAIAgent",
    model=openai,
    tools=web_search_tool_kit() + commomon_tool_kit(),
    system_prompt="你是一个智能助理，能够使用提供的工具来回答用户的问题，优先考虑英文搜索结果和搜索引擎吧，search tool提供的是searxng",
)


In [ ]:
ds_agent.invoke({
    "messages" : [
        {"role": "user", "content": "最近有个叫做claw bot的项目挺火的，你看看能帮我了解一下它的主要内容吗？不要再反问我问题了"}]
},
config={"callbacks": [CallbackHandler()]}
)